<h1>Spark basics practice</h1>
<li>The code in the next cell extracts covid data from New York State's covid repository</li>
<li>The extracted data is stored in an RDD containing an Array of (String, String,Int,Int) matching (date, borough, positive cases, tests) for each day since March 1st 2020 (the data is ordered by time</li>
<li>Use this RDD to answer the questions below</li>

In [1]:
val counties = Array("Kings","Queens","New+York","Suffolk","Bronx","Nassau","Westchester","Erie",
                     "Monroe","Richmond","Onondaga","Orange","Rockland","Albany","Dutchess",
                     "Saratoga","Oneida","Niagara","Broome","Ulster","Rensselaer","Schenectady",
                     "Chautauqua","Oswego","Jefferson","Ontario","St.+Lawrence","Tompkins",
                     "Putnam","Steuben","Wayne","Chemung","Sullivan","Clinton","Cattaraugus",
                     "Cayuga","Madison","Warren","Columbia","Livingston","Washington","Herkimer",
                     "Otsego","Genesee","Fulton","Montgomery","Greene","Tioga","Franklin","Chenango",
                     "Cortland","Allegany","Delaware","Wyoming","Orleans"
                     ,"Essex","Seneca","Schoharie","Lewis","Yates","Schuyler","Hamilton")

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.7.189:4041
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1665244576771)
SparkSession available as 'spark'


counties: Array[String] = Array(Kings, Queens, New+York, Suffolk, Bronx, Nassau, Westchester, Erie, Monroe, Richmond, Onondaga, Orange, Rockland, Albany, Dutchess, Saratoga, Oneida, Niagara, Broome, Ulster, Rensselaer, Schenectady, Chautauqua, Oswego, Jefferson, Ontario, St.+Lawrence, Tompkins, Putnam, Steuben, Wayne, Chemung, Sullivan, Clinton, Cattaraugus, Cayuga, Madison, Warren, Columbia, Livingston, Washington, Herkimer, Otsego, Genesee, Fulton, Montgomery, Greene, Tioga, Franklin, Chenango, Cortland, Allegany, Delaware, Wyoming, Orleans, Essex, Seneca, Schoharie, Lewis, Yates, Schuyler, Hamilton)


In [2]:
//DON'T CHANGE THIS CODE!!!!

val counties = Array("Kings","Queens","New+York","Suffolk","Bronx","Nassau","Westchester","Erie",
                     "Monroe","Richmond","Onondaga","Orange","Rockland","Albany","Dutchess",
                     "Saratoga","Oneida","Niagara","Broome","Ulster","Rensselaer","Schenectady",
                     "Chautauqua","Oswego","Jefferson","Ontario","St.+Lawrence","Tompkins",
                     "Putnam","Steuben","Wayne","Chemung","Sullivan","Clinton","Cattaraugus",
                     "Cayuga","Madison","Warren","Columbia","Livingston","Washington","Herkimer",
                     "Otsego","Genesee","Fulton","Montgomery","Greene","Tioga","Franklin","Chenango",
                     "Cortland","Allegany","Delaware","Wyoming","Orleans"
                     ,"Essex","Seneca","Schoharie","Lewis","Yates","Schuyler","Hamilton")

val base_url = "https://health.data.ny.gov/resource/xdss-u53e.json?County="
val urls = counties.map(a => base_url+a) //Makes a url for each county

//This gets the contents of the url
//results is an array with one entry per county
//the data for each county is in JSON format
val results = urls.map(u => scala.io.Source.fromURL(u).mkString) 

//Create an rdd in 32 partitions (there is a lot of data)
//Reads the json and converts it into a spark dataframe (we'll do this later)
//and then converts the df into an rdd
//finally, extract the date, the county name, the new cases on that date and tests done
//on that date
val data_rdd = spark.read.json(sc.parallelize(results,32))
                    .rdd
                    .map(r => (r(5).toString.slice(0,10), 
                               r(0).toString,
                               r(4).toString.toInt,
                               r(7).toString.toInt))


counties: Array[String] = Array(Kings, Queens, New+York, Suffolk, Bronx, Nassau, Westchester, Erie, Monroe, Richmond, Onondaga, Orange, Rockland, Albany, Dutchess, Saratoga, Oneida, Niagara, Broome, Ulster, Rensselaer, Schenectady, Chautauqua, Oswego, Jefferson, Ontario, St.+Lawrence, Tompkins, Putnam, Steuben, Wayne, Chemung, Sullivan, Clinton, Cattaraugus, Cayuga, Madison, Warren, Columbia, Livingston, Washington, Herkimer, Otsego, Genesee, Fulton, Montgomery, Greene, Tioga, Franklin, Chenango, Cortland, Allegany, Delaware, Wyoming, Orleans, Essex, Seneca, Schoharie, Lewis, Yates, Schuyler, Hamilton)
base_url: String = https://health.data.ny.gov/resource/xdss-u53e.json?County=
urls: Array[String] = Array(https://health.data.ny.gov/resource/xdss-u53e.json?County=Kings, https://health.d...


<h1>Question 1</h1>
<li>Using <span style="color:blue">reduce</span> calculate the total number of cases and total number of tests in New York State</li>

In [3]:
val total_cases_tests = data_rdd.map(r=> (r._3,r._4))
                                .reduce((a,b) => (a._1+b._1,a._2+b._2))
//As of 10/3/2022, the result was: (6055593,119988852)

total_cases_tests: (Int, Int) = (6071907,120210799)


<h1>Question 2</h1>
Using <span style="color:blue">reduceByKey</span> calculate the number of cases and total number of tests by county

Result as of 10/3/2022:
<pre>
cases_tests_by_county: Array[(String, (Int, Int))] = Array((Oneida,(64524,1496227)), (Tompkins,(24729,3050525)), (Chemung,(24817,469310)), (Schenectady,(40642,818993)), (Cattaraugus,(18139,301417)), (Greene,(10155,178413)), (Wyoming,(9412,154961)), (Columbia,(12685,249434)), (Chenango,(11043,231664)), (Ulster,(39314,849264)), (Clinton,(20762,398096)), (Wayne,(20377,372007)), (Herkimer,(16385,317604)), (Nassau,(498703,8184078)), (Seneca,(7049,135717)), (Lewis,(6920,102023)), (Broome,(54347,1155989)), (Erie,(252478,4154077)), (Livingston,(13481,279849)), (Bronx,(465438,8622412)), (Allegany,(10188,244807)), (Queens,(785765,14357698)), (Jefferson,(24654,394596)), (Orleans,(9907,154881)), (Putnam,(28794,477768)), (Rensselaer,(38906,859451)), (Ontario,(24127,468338)), (Suffolk,(515835,8275540...
</pre>

In [4]:
val cases_tests_by_county =data_rdd.map(r => (r._2,(r._3,r._4))).reduceByKey((a,b) => (a._1 + b._1, a._2 + b._2)).collect

cases_tests_by_county: Array[(String, (Int, Int))] = Array((Oneida,(64808,1499245)), (Tompkins,(24801,3051374)), (Chemung,(24911,470265)), (Schenectady,(40780,820520)), (Cattaraugus,(18193,301842)), (Greene,(10174,178688)), (Wyoming,(9444,155300)), (Columbia,(12727,249843)), (Chenango,(11075,232217)), (Ulster,(39450,850804)), (Clinton,(20864,398672)), (Wayne,(20438,372829)), (Herkimer,(16451,318330)), (Nassau,(499938,8199394)), (Seneca,(7079,136001)), (Lewis,(6939,102200)), (Broome,(54548,1158607)), (Erie,(253253,4161555)), (Livingston,(13520,280316)), (Bronx,(466374,8639243)), (Allegany,(10208,245125)), (Queens,(787651,14386172)), (Jefferson,(24780,395770)), (Orleans,(9938,155168)), (Putnam,(28875,478756)), (Rensselaer,(39030,860976)), (Ontario,(24227,469400)), (Suffolk,(517298,8293278...


<h1>Question 3</h1>
Using <span style="color:blue">reduceByKey</span> calculate the positivity by month for New York State. Then use <span style="color:blue">Math.round</span>, <span style="color:blue">sortBy</span>, and <span style="color:blue">collect</span> to display your results as a percentage with 2 decimal places. A sample of the expected result (the percentage for 10/2022 will change depending on when you pull the data):
<pre>
(2020-03,37.89)
(2020-04,31.79)
(2020-05,5.34)
(2020-06,1.2)
(2020-07,1.08)
(2020-08,0.83)
(2020-09,0.97)
(2020-10,1.3)
(2020-11,2.98)
(2020-12,5.65)
(2021-01,6.27)
(2021-02,3.66)
(2021-03,3.35)
(2021-04,2.69)
(2021-05,1.09)
(2021-06,0.42)
(2021-07,1.56)
(2021-08,3.15)
(2021-09,2.9)
(2021-10,2.31)
(2021-11,3.55)
(2021-12,10.54)
(2022-01,15.09)
(2022-02,2.93)
(2022-03,1.95)
(2022-04,5.45)
(2022-05,7.87)
(2022-06,5.88)
(2022-07,9.54)
(2022-08,7.08)
(2022-09,6.94)
(2022-10,6.71)
</pre>

In [5]:
val cases_tests_by_month = data_rdd
.map(r => (r._1.slice(0,7),(r._3,r._4)))
.reduceByKey((a,b) => (a._1 + b._1, a._2 + b._2))//.sortBy(_._2._1)

cases_tests_by_month: org.apache.spark.rdd.RDD[(String, (Int, Int))] = ShuffledRDD[18] at reduceByKey at <console>:24


In [6]:
cases_tests_by_month
    .map(t => (t._1,Math.round(t._2._1.toDouble/t._2._2*10000)/100.0))
    .sortBy(t=>t._1).collect.foreach(println)

(2020-03,37.89)
(2020-04,31.79)
(2020-05,5.34)
(2020-06,1.2)
(2020-07,1.08)
(2020-08,0.83)
(2020-09,0.97)
(2020-10,1.3)
(2020-11,2.98)
(2020-12,5.65)
(2021-01,6.27)
(2021-02,3.66)
(2021-03,3.35)
(2021-04,2.69)
(2021-05,1.09)
(2021-06,0.42)
(2021-07,1.56)
(2021-08,3.15)
(2021-09,2.9)
(2021-10,2.31)
(2021-11,3.55)
(2021-12,10.54)
(2022-01,15.09)
(2022-02,2.93)
(2022-03,1.95)
(2022-04,5.45)
(2022-05,7.87)
(2022-06,5.88)
(2022-07,9.54)
(2022-08,7.08)
(2022-09,6.94)
(2022-10,7.16)


<h1>Question 4</h1>
Return an RDD of (date,county,positivity) where positivity is the percentage of tests that are positive. For this problem, you must use the Option case class to handle the case where the divisor is zero

Result as of 10/3/2022:

<pre>
res0: Array[(String, String, Double)] = Array((2022-10-02,Kings,4.0), (2022-10-01,Kings,3.0), (2022-09-30,Kings,4.0), (2022-09-29,Kings,5.0), (2022-09-28,Kings,5.0), (2022-09-27,Kings,4.0), (2022-09-26,Kings,3.0), (2022-09-25,Kings,5.0), (2022-09-24,Kings,4.0), (2022-09-23,Kings,3.0), (2022-09-22,Kings,4.0), (2022-09-21,Kings,4.0), (2022-09-20,Kings,3.0), (2022-09-19,Kings,7.0), (2022-09-18,Kings,5.0), (2022-09-17,Kings,3.0), (2022-09-16,Kings,5.0), (2022-09-15,Kings,4.0), (2022-09-14,Kings,3.0), (2022-09-13,Kings,3.0), (2022-09-12,Kings,6.0), (2022-09-11,Kings,5.0), (2022-09-10,Kings,5.0), (2022-09-09,Ki...
</pre>

In [1]:
def divide(t: (String,String,Int,Int)): Option[(String,String,Double)] = {
    try {
        Some((t._1,t._2,100*t._3/t._4))
    } catch {
        case e: Exception => None
    }
}
val positivity = data_rdd.map(t => divide(t)).flatMap(e=>e)
positivity.collect

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.149:4051
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1666146871200)
SparkSession available as 'spark'


<console>: 30: error: not found: value data_rdd

<h1>Question 5</h1>
Return the tuple (date,county,positivity) where the positivity was the highest (use <span style="color:blue">takeOrdered</span>)

In [ ]:
val highest_positive = positivity.takeOrdered(1)(Ordering[Double].on(x => -1*x._3))